In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from tqdm import tqdm

This notebook demonstrates how the optimal HFX values (i.e., those that minimize error between DLPNO-CCSD(T) and a HFX-adjusted semilocal functional using that amount of HFX) were found for each structure.

# CSD-76-HFX

In [2]:
sse_df = pd.read_csv('../data/cleaned_csd76_sse.csv').set_index('Unnamed: 0')

complexes = sse_df.index
functionals = sse_df.columns

In [3]:
#determine the optimal HFX amount

csd_76 = pd.read_csv('../data/CSD-76.csv').set_index('name')

def correct_hfx(structure, functional):
    df = csd_76
    reference = df.loc[structure]['dlpno-CCSD_T.vertsse']

    sses = []
    all_increments = np.arange(0, 101, 5)
    sse_ref = sse_df
    increments = []
    for increment in all_increments:
        sse = sse_ref.loc[structure][functional + '_hfx_' + str(increment)]
        if not np.isnan(sse):
            increments.append(increment)
            sses.append(sse)
    if len(increments) < 5:
        #print('Not enough converged values!')
        return np.nan, np.nan
        
    fit = interp1d(increments, sses, kind='linear', fill_value='extrapolate')
    sse_intersection = lambda hfx: (fit(hfx) - reference)**2
    res = minimize(sse_intersection, 25, method='Nelder-Mead')#, bounds=((0,100),))
    if res.success:
        min_error = res.fun
        result = res.x[0]
    else:
        print(idx, res)
    #try other initial guesses to see if we can improve
    for guess in [0, 50, 75, 100]:
        res = minimize(sse_intersection, guess, method='Nelder-Mead')#, bounds=((0,100),))
        if res.success:
            #if we found a better solution (plus some amount to prevent going away from a found zero)
            if res.fun < min_error - 0.1 and res.x[0] >=0 and res.x[0] <= 100:
                result = res.x[0]
                min_error = res.fun
        else:
            print(idx, res)
    extrapolation = False
    if result < increments[0] or result > increments[-1]:
        extrapolation = True
    return result, extrapolation

hfx_df = pd.DataFrame(index=complexes, columns=['hfx_pbe', 'hfx_scan'])

pbe_extrapolate = 0
scan_extrapolate = 0
for name in tqdm(complexes):
    a, b = correct_hfx(name, 'pbe')
    hfx_df['hfx_pbe'][name] = a
    if not np.isnan(b):
        pbe_extrapolate += b
    a, b = correct_hfx(name, 'scan')
    hfx_df['hfx_scan'][name] = a
    if not np.isnan(b):
        scan_extrapolate += b
    

print(f'The number of structures without sufficient points for PBE is {np.sum([np.isnan(x) for x in hfx_df["hfx_pbe"].to_numpy()])}.')
print(f'The number of structures without sufficient points for SCAN is {np.sum([np.isnan(x) for x in hfx_df["hfx_scan"].to_numpy()])}.')

print(f'The number of structures which required extrapolation for PBE is {pbe_extrapolate}.')
print(f'The number of structures which required extrapolation for SCAN is {scan_extrapolate}.')

hfx_df.to_csv('../data/CSD76targets.csv')

100%|███████████████████████████████████████████| 72/72 [00:04<00:00, 15.08it/s]

The number of structures without sufficient points for PBE is 3.
The number of structures without sufficient points for SCAN is 5.
The number of structures which required extrapolation for PBE is 2.
The number of structures which required extrapolation for SCAN is 2.


In [4]:
hfx_df
old_df = pd.read_csv('../data/CSD76targets.csv').set_index('Unnamed: 0')

for idx, row in hfx_df.iterrows():
    if not np.isclose(row['hfx_pbe'], old_df.loc[idx]['hfx_pbe'], equal_nan=True):
        print(idx, row['hfx_pbe'], old_df.loc[idx]['hfx_pbe'])
    if not np.isclose(row['hfx_scan'], old_df.loc[idx]['hfx_scan'], equal_nan=True):
        print(idx, row['hfx_scan'], old_df.loc[idx]['hfx_scan'])

# VSS-452

In [5]:
sse_df = pd.read_csv('../data/cleaned_vss452_sse.csv').set_index('Unnamed: 0')

complexes = sse_df.index
functionals = sse_df.columns

In [6]:
vss_452 = pd.read_csv('../data/VSS-452.csv')
vss_452 = vss_452.set_index(vss_452['name'])

def correct_hfx(structure, functional):
    df = vss_452
    reference = df.loc[structure.split('/')[-1]]['dlpno-CCSD_T.vertsse']

    sses = []
    all_increments = np.arange(0, 101, 5)
    sse_ref = sse_df
    increments = []
    for increment in all_increments:
        sse = sse_ref.loc[structure][functional + '_hfx_' + str(increment)]
        if not np.isnan(sse):
            increments.append(increment)
            sses.append(sse)
    if len(increments) < 5:
        #print('Not enough converged values!')
        return np.nan, np.nan
        
    fit = interp1d(increments, sses, kind='linear', fill_value='extrapolate')
    sse_intersection = lambda hfx: (fit(hfx) - reference)**2
    res = minimize(sse_intersection, 25, method='Nelder-Mead')#, bounds=((0,100),))
    if res.success:
        min_error = res.fun
        result = res.x[0]
    else:
        print(idx, res)
    #try other initial guesses to see if we can improve
    for guess in [0, 50, 75, 100]:
        res = minimize(sse_intersection, guess, method='Nelder-Mead')#, bounds=((0,100),))
        if res.success:
            #if we found a better solution (plus some amount to prevent going away from a found zero)
            if res.fun < min_error - 0.1 and res.x[0] >=0 and res.x[0] <= 100:
                result = res.x[0]
                min_error = res.fun
        else:
            print(idx, res)
    extrapolation = False
    if result < increments[0] or result > increments[-1]:
        extrapolation = True
    return result, extrapolation

hfx_df = pd.DataFrame(index=complexes, columns=['hfx_pbe', 'hfx_scan'])

pbe_extrapolate = 0
scan_extrapolate = 0
for name in tqdm(complexes):
    a, b = correct_hfx(name, 'pbe')
    hfx_df['hfx_pbe'][name] = a
    if not np.isnan(b):
        pbe_extrapolate += b
    a, b = correct_hfx(name, 'scan')
    hfx_df['hfx_scan'][name] = a
    if not np.isnan(b):
        scan_extrapolate += b
    

print(f'The number of structures without sufficient points for PBE is {np.sum([np.isnan(x) for x in hfx_df["hfx_pbe"].to_numpy()])}.')
print(f'The number of structures without sufficient points for SCAN is {np.sum([np.isnan(x) for x in hfx_df["hfx_scan"].to_numpy()])}.')

print(f'The number of structures which required extrapolation for PBE is {pbe_extrapolate}.')
print(f'The number of structures which required extrapolation for SCAN is {scan_extrapolate}.')

hfx_df.to_csv('../data/VSS452targets.csv')

100%|█████████████████████████████████████████| 452/452 [00:28<00:00, 16.09it/s]

The number of structures without sufficient points for PBE is 47.
The number of structures without sufficient points for SCAN is 61.
The number of structures which required extrapolation for PBE is 33.
The number of structures which required extrapolation for SCAN is 51.
